In [1]:
import pandas as pd
import sqlite3

class CreateGapminderDB:
    def __init__(self):
        self.file_names = [
                            "ddf--datapoints--gdp_pcap--by--country--time.csv",
                            "ddf--datapoints--lex--by--country--time.csv",
                            "ddf--datapoints--pop--by--country--time.csv",
                            "ddf--entities--geo--country.csv"
                            ]   

        self.table_names = [
                            "gdp_per_capita",
                            "life_expectancy",
                            "population",
                            "geography"
                            ]

    def import_as_dataframe(self):
        df_dict = {}
        for file_name, table_name in zip(self.file_names, self.table_names):
            file_path = f"data/{file_name}"
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Append the DataFrame to the list
            df_dict[table_name] = df
            print(f"Loaded {file_path}- {len(df_dict)}")
        return df_dict
   
    # Create a database named gapminder.db in SQLlite a plotting with the data
    def create_database(self):
    
        ## Start a connection to the SQLite database
        connection = sqlite3.connect("data/gapminder.db")
        
        df_dict = self.import_as_dataframe()
        for key, value in df_dict.items():
            value.to_sql(name=key, con=connection, index=False, if_exists="replace")
        
        ## Send the SQL descriptions to SQLite database
        
        ### Ensure that there's no plot with the same name we are going to create
        drop_view_sql = """
        DROP VIEW IF EXISTS plotting;
        """

        create_view_sql = """
        CREATE VIEW plotting AS
        SELECT  geography.name AS country_name,
                geography.world_4region As continent,
                gdp_per_capita.time AS dt_year, 
                gdp_per_capita.gdp_pcap AS gdp_per_capita,
                life_expectancy.lex As life_expectancy,
                population.pop AS population
                
            FROM    gdp_per_capita
            JOIN    geography
            ON      gdp_per_capita.country = geography.country
            JOIN    life_expectancy
            ON      gdp_per_capita.country = life_expectancy.country AND
                    gdp_per_capita.time = life_expectancy.time
            JOIN    population
            ON      gdp_per_capita.country = population.country AND
                    gdp_per_capita.time = population.time
            WHERE   gdp_per_capita.time <2024; 
        """

        ### Execute the SQL script
        cur = connection.cursor()
        cur.execute(drop_view_sql)
        cur.execute(create_view_sql)

        connection.close()
    
    

In [2]:
create_gapminder_db = CreateGapminderDB()
create_gapminder_db.create_database()

Loaded data/ddf--datapoints--gdp_pcap--by--country--time.csv- 1
Loaded data/ddf--datapoints--lex--by--country--time.csv- 2
Loaded data/ddf--datapoints--pop--by--country--time.csv- 3
Loaded data/ddf--entities--geo--country.csv- 4
